# 파이토치의 기본 자료형인 Tensor에 대해 알아보자.# 

In [54]:
import torch
import numpy as np

간단하게 torch.Tensor라는 명령어를 통해서 Tensor를 만들 수 있다.

In [39]:
a = torch.tensor([[1,2,3],[4,5,6]]) # 이렇게 2d array를 Tensor로 만들어 줄 수도 있다.
print(a)

tensor([[1., 2., 3.],
        [4., 5., 6.]])


In [40]:
b = torch.tensor([[1,2,3],[4,5,6]], dtype=torch.float) # data type을 임의로 지정해줄 수 있다. 보통은 알아서 추론 한다.
print(b)

tensor([[1., 2., 3.],
        [4., 5., 6.]])


In [45]:
cuda0 = torch.device('cuda:0')
c = torch.tensor([1.0,2.0,3.0], device=cuda0) # Tensor가 어떤 Device에서 작동할 건지 지정해줄 수 있다. 보통은 한꺼번에 지정을 한다.
print(c)

tensor([1., 2., 3.], device='cuda:0')


In [51]:
# dtype을 지정하지 않았을 때, default는 무엇일까?
# 그리고 float이라고만 쓴 y의 경우에는 float32일까 float 64일까?
print(a.dtype)
print(b.dtype)
print(c.dtype)

torch.int64
torch.float32
torch.float32


### 결과 ###
- int의 기본값은 int64
- float의 기본값은 float32

In [57]:
# numpy의 그것처럼 zeros, ones 등이 존재한다.
zeros= torch.zeros((1,2,), dtype=torch.int)
print(zeros)

tensor([[0, 0]], dtype=torch.int32)


In [58]:
# numpy array를 통해서도 만들 수 있다.
np_zeros = np.zeros((1,2))
print(np_zeros)
zeros_from_np = torch.tensor(np_zeros)
print(zeros_from_np)

[[0. 0.]]
tensor([[0., 0.]], dtype=torch.float64)


### requires_grad ###

파이토치가 다른 프레임워크가 가장 차별화가 되는 지점이 바로 이 requires_grad라는 옵션이라고 생각한다.  
**Tensor에 이 옵션을 부여하면 이 Tensor에, 혹은 이 Tensor를 가지고 행해지는 operation들을 저장하고 있다가 나중에 gradient를 계산할 때 알아서 backprogation을 해준다.**  
즉, 우리가 x라는 tensor에 어떤 operation을 행했고, 그리고 gradient를 구하기 위해서 미분을 했을 경우에 어떻게 값이 나오는지에 대한 식을 직접 만들 필요가 없다는 점이다.  

다음의 예제에서 우리가 하고 싶은 것은 다음과 같다.  
1. X라는 input이 있다.  
2. X라는 matrix는 여러 operation을 거쳐 Z라는 matrix가 된다.  
3. 우리가 가지고 있는 target값이 있어서 각 Z와 target의 차이의 합을 loss로 정의한다.  
4. **(가장 중요한 것)** 특정 X값 ([2,3])과 target값([3,4])에 대한 loss를 각 X의 element (x0, x1)으로 편미분한 값을 계산한다.  
  
4번이 중요한 이유는 이 같은 편미분값이 우리가 흔히 말하는 Gradient Descent가 되기 때문이고 Neural network를 학습시킬 때 기본적으로 필요한 값이기 때문이다.

In [60]:
X = torch.tensor([2.0, 3.0], requires_grad=True)
Y = X ** 2
Z = 2 * Y + 3

target = torch.tensor([3.0, 4.0])
loss = torch.sum(torch.abs(Z-target))
loss.backward()

print(X.grad, Y.grad, Z.grad)

tensor([ 8., 12.]) None None
<ipython-input-60-042d385d256d>:9: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  print(x.grad, y.grad, z.grad)


직접 chain rule을 이용해 d_loss / d_x0 을 계산해보자면 다음과 같다.  
  
loss = (z0 - target0) + (z1 - target1) ==> d_loss / d_z0 = 1  
z_0 = 2 * y0 + 3 ==> d_z0 / d_y0 = 2  
y0 = x0 ** 2 ==> d_y0 / d_x0 = 2 * x0  
  
주어진 값을 가지고 위의 미분값들을 다 곱해보면 최종 결과가 나온다.  
d_loss / d_x0 = (1) * (2) * (2 * 2) = 8
  
pytorch를 이용해 구한 x.grad의 첫번째 값과 같은 것을 확인할 수 있다.  
**다시 한번 말하지만 pytorch가 좋은 점은 우리가 직접 계산한 것처럼 미분식을 analytical하게 일일이 구하지 않아도 된다는 점이다.**

In [61]:
# 우리가 만약 Y와 Z의 grad도 필요했다면 다음과 같이 하면 될 것이다.
X = torch.tensor([2.0, 3.0], requires_grad=True)
Y = X ** 2
Z = 2 * Y + 3

Y.retain_grad()
Z.retain_grad()

target = torch.tensor([3.0, 4.0])
loss = torch.sum(torch.abs(Z-target))
loss.backward()

print(X.grad, Y.grad, Z.grad)

tensor([ 8., 12.]) tensor([2., 2.]) tensor([1., 1.])


꼭 기억해야 할 점은 우리는 loss.backward()란 명령어로 grad들을 구했기 때문에  
우리가 chain rule로 구했던 것처럼 dz / dy 같은 중간 미분값을 구하는 것이 아니라  
우모든 값들이 loss를 편미분한 것이다.  